In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
import os
import numpy as np
import crocoddyl as c
import numdifftools as nd
from time import perf_counter
c.switchToNumpyArray()

import matplotlib.pyplot as plt
from scipy.interpolate import griddata
from scipy.stats import kde
import torch
from irepa import *

In [3]:
p1 = np.loadtxt('irepa_run1.out', delimiter = ",")
p5 = np.loadtxt('irepa_run4.out', delimiter = ",")
p10 = np.loadtxt('irepa_run10.out', delimiter = ",")
p15 = np.loadtxt('irepa_run15.out', delimiter = ",")
p20 = np.loadtxt('irepa_run20.out', delimiter = ",")


In [4]:
def plot_cost(x, y, z, n = 1):

    extent = (min(x), max(x), min(y), max(y))
    xs,ys = np.mgrid[extent[0]:extent[1]:100j, extent[2]:extent[3]:200j] #2D x,y
    z_resampled = griddata((x, y), z, (xs, ys)) #2D z
    plt.imshow(z_resampled, interpolation="none", extent=extent, origin="lower")
    plt.xticks(np.arange(-3, 3.1, step=1))
    plt.yticks(np.arange(-3, 3.1, step=1))
    clb = plt.colorbar()
    clb.set_label('Cost', labelpad=-40, y=1.10, rotation=0)
    plt.savefig(f"./Images/Cost{n}.png")
    plt.close()

    
def plot_iters(x, y, z, n = 1):
 
    extent = (min(x), max(x), min(y), max(y))
    xs,ys = np.mgrid[extent[0]:extent[1]:100j, extent[2]:extent[3]:200j] #2D x,y
    z_resampled = griddata((x, y), z, (xs, ys)) #2D z
    plt.imshow(z_resampled, interpolation="none", extent=extent, origin="lower")
    plt.xticks(np.arange(-3, 3.1, step=1))
    plt.yticks(np.arange(-3, 3.1, step=1))
    clb = plt.colorbar()
    clb.set_label('Iterations', labelpad=-40, y=1.10, rotation=0)
    plt.savefig(f"./Images/ddp_iters{n}.png")
    plt.close()

In [5]:
x1, y1, cost1, iters1 = p1[:,0], p1[:,1], p1[:,2], p1[:,3] 

x5, y5, cost5, iters5 = p5[:,0], p5[:,1], p5[:,2], p5[:,3] 

x10, y10, cost10, iters10 = p10[:,0], p10[:,1], p10[:,2], p10[:,3] 

x15, y15, cost15, iters15 = p15[:,0], p15[:,1], p15[:,2], p15[:,3] 

x20, y20, cost20, iters20 = p20[:,0], p20[:,1], p20[:,2], p20[:,3] 

In [6]:
plot_cost(x20, y20, cost20, n=20)
plot_iters(x20, y20, iters20, n=20)


plot_cost(x10, y10, cost10, n=10)
plot_iters(x10, y10, iters10, n= 10)

plot_cost(x15, y15, cost15, n=15)
plot_iters(x15, y15, iters15, n=15)

plot_cost(x5, y5, cost5, n = 15)
plot_iters(x5, y5, iters5, n = 15)

plot_cost(x1, y1, cost1, n = 1)
plot_iters(x1, y1, iters1, n =1)

In [ ]:

    
def croc():
    # get the cost
    # X, Y, cost, iters 
    data = []
    model = c.ActionModelUnicycle()
    for _ in range(1000):

        x0 = np.array([np.random.uniform(-2.1, 2.1), np.random.uniform(-2.1, 2.1), np.random.uniform(0,1)])
        T = 30
        problem = c.ShootingProblem(x0.T, [ model ] * T, model)
        ddp = c.SolverDDP(problem)
        ddp.solve([], [], 1000)
        data.append([float(x0[0]), float(x0[1]), ddp.cost, ddp.iter])
    return np.asarray(data)

def scatter_cost(data, n = 0, cmap = 'viridis'):
    plt.figure(figsize=(4,4))
    plt.scatter(data[:,0], data[:,1], c = data[:,2], cmap = cmap)
    clb = plt.colorbar()
    plt.grid()

    clb.set_label('Cost',  y=-0.01, rotation=0)
    plt.title(f"Irepa {n}th iteration")
    #plt.savefig(f"./images/Irepa {n}th run cost.png")


def scatter_iters(data, n= 0, cmap = "viridis"):

    plt.figure(figsize=(4,4))
    plt.scatter(data[:,0], data[:,1], c = data[:,3], cmap = cmap)
    clb = plt.colorbar()
    plt.grid()

    clb.set_label('Iterations',  y=-0.01, rotation=0)
    plt.title(f"Irepa {n}th iteration")
    #plt.savefig(f"./images/Irepa {n}th run iterations.png")


In [ ]:
data = croc()

In [ ]:
scatter_cost(data, n = 0)
scatter_iters(data, n = 0)

scatter_cost(run1, n = 1)
scatter_iters(run1, n = 1)

scatter_cost(run4, n = 4)
scatter_iters(run4, n = 4)

scatter_cost(run10, n = 10)
scatter_iters(run10, n = 10)

scatter_cost(run15, n = 15)
scatter_iters(run15, n = 15)

scatter_cost(run20, n = 20)
scatter_iters(run20, n = 20)